# Flare Detector Template (EK Dra)

このノートブックでは `src.flarepy_EK_Dra.FlareDetector_EK_Dra` の基本的な使い方を示します。


## 1. Setup

Plotly Express と Graph Objects の両方を使用できるように準備します。


In [ ]:
import os
import sys
import re
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

NOTEBOOK_DIR = Path().resolve()
PROJECT_ROOT = NOTEBOOK_DIR.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
print('PROJECT_ROOT:', PROJECT_ROOT)


## 2. FITS ファイルの確認


In [ ]:
DATA_DIR = PROJECT_ROOT / 'data' / 'TESS' / 'EK_Dra'
fits_files = sorted(DATA_DIR.glob('*.fits'))
print('found', len(fits_files), 'files')
fits_files[:3]


## 3. FlareDetector_EK_Dra で処理


In [ ]:
from src.flarepy_EK_Dra import FlareDetector_EK_Dra

example_file = fits_files[0] if fits_files else None
detector = None
if example_file is not None:
    detector = FlareDetector_EK_Dra(file=str(example_file), process_data=True)
detector


## 4. サマリ


In [ ]:
summary = {
    'file': detector.file,
    'flare_number': detector.flare_number,
    'sum_flare_energy': detector.sum_flare_energy,
    'precise_obs_time': detector.precise_obs_time,
    'flare_ratio': getattr(detector, 'flare_ratio', None),
}
summary


## 5. Plotly Express: Light Curve


In [ ]:
px.line(
    x=detector.tessBJD,
    y=detector.mPDCSAPflux,
    labels={'x': 'BJD - 2457000', 'y': 'Normalized Flux'},
    title='EK Dra Light Curve (Plotly Express)'
)


## 6. Plotly Express: Energy vs Time


In [ ]:
px.scatter(
    x=detector.tessBJD,
    y=detector.energy,
    labels={'x': 'BJD - 2457000', 'y': 'Energy'},
    title='Flare Energy vs Time (Plotly Express)'
)


## 7. Plotly Express: Flare Energy Histogram


In [ ]:
px.histogram(
    x=detector.energy,
    nbins=50,
    labels={'x': 'Energy', 'y': 'Count'},
    title='Flare Energy Distribution (Plotly Express)'
)


## 8. Plotly Graph Objects: Light Curve


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=detector.tessBJD,
    y=detector.mPDCSAPflux,
    mode='lines',
    name='Normalized Flux'
))
fig.update_layout(
    title='EK Dra Light Curve (Graph Objects)',
    xaxis_title='BJD - 2457000',
    yaxis_title='Normalized Flux'
)
fig.show()


## 9. 複数ファイルの処理例


In [ ]:
detectors = []
for file_path in fits_files[:3]:
    det = FlareDetector_EK_Dra(file=str(file_path), process_data=True)
    detectors.append(det)
print('processed', len(detectors), 'files')


## 10. 追加解析


In [ ]:
print('Rotation period:', detector.per)
print('First 5 flare energies:', detector.energy[:5])
px.box(
    y=[d.sum_flare_energy for d in detectors],
    labels={'y': 'Sum Flare Energy'},
    title='Sum Flare Energy per File'
)
